## Reading JSON Data in PySpark

**Potential interview question**
- What is JSON data and how to read it in Spark?
- What if I have 3 keys in all lines and 4 keys in one line?
- What is multiline and line-delimited JSON?
- Which one works faster — multiline or line-delimited?
- How to convert nested JSON into Spark DataFrame?
- What will happen if I have a corrupted JSON file?


### What is JSON data and how to read it in Spark?

- `JSON (JavaScript Object Notation):` A lightweight data-interchange format that is easy to read and write for humans and machines.
- `Structure:` Data is stored as key-value pairs in a hierarchical or nested format.
- Example: `{"id": 1, "name": "Alice", "age": 25}`

#### How to Read JSON in Spark?
- Spark provides built-in support to read JSON files as DataFrames.
- We can use `spark.read.json` or we can use generic `spark.read.format` method as well
- Options:
  - `multiline:` Set True for multiline JSON.
  - `schema:` Use a predefined schema for performance optimization.




**Line-Delimited JSON:**

In [0]:
spark.read.format("json") \
    .option("inferSchema", "true") \
    .option("mode", "permissive") \
    .load("dbfs:/FileStore/shared_uploads/zaderohish5@gmail.com/line_delimited_json.json").show()

+---+--------+------+
|age|    name|salary|
+---+--------+------+
| 20|  Manish| 20000|
| 25|  Nikita| 21000|
| 16|  Pritam| 22000|
| 35|Prantosh| 25000|
| 67|  Vikash| 40000|
+---+--------+------+



### What if I have 3 keys in all lines and 4 keys in one line?
When reading JSON with inconsistent keys:
- Spark automatically handles this by creating a column for every unique key.
- Rows missing values for certain keys will have NULL in those columns.


In [0]:
# Gender is a extra column for Vikas, for other spark assigned null
spark.read.format("json") \
    .option("inferSchema", "true") \
    .option("mode", "permissive") \
    .load("dbfs:/FileStore/shared_uploads/zaderohish5@gmail.com/single_file_json_with_extra_fields.json").show()

+---+------+--------+------+
|age|gender|    name|salary|
+---+------+--------+------+
| 20|  null|  Manish| 20000|
| 25|  null|  Nikita| 21000|
| 16|  null|  Pritam| 22000|
| 35|  null|Prantosh| 25000|
| 67|     M|  Vikash| 40000|
+---+------+--------+------+



### What is multiline and line-delimited JSON?

**Line-Delimited JSON:**
- Each JSON object is on a single line.
- Example:
  ```json
  {"id": 1, "name": "Alice"}
  {"id": 2, "name": "Bob"}
  ```

**Multiline JSON:**
- A single JSON object spans multiple lines.
  - Example:
  ```json
  {
    "id": 1,
    "name": "Alice"
  }
  {
    "id": 2,
    "name": "Bob"
  }
  ```

#### How to Handle in Spark:

**For line-delimited JSON (default):**

In [0]:
spark.read.format("json") \
    .option("inferSchema", "true") \
    .option("mode", "permissive") \
    .load("dbfs:/FileStore/shared_uploads/zaderohish5@gmail.com/line_delimited_json.json").show()

+---+--------+------+
|age|    name|salary|
+---+--------+------+
| 20|  Manish| 20000|
| 25|  Nikita| 21000|
| 16|  Pritam| 22000|
| 35|Prantosh| 25000|
| 67|  Vikash| 40000|
+---+--------+------+



**For multiline JSON:**
- multiline: Set True for multiline JSON.

In [0]:
spark.read.format("json") \
    .option("inferSchema", "true") \
    .option("mode", "permissive") \
    .option("multiline", "true") \
    .load("dbfs:/FileStore/shared_uploads/zaderohish5@gmail.com/Multi_line_correct.json").show()

+---+--------+------+
|age|    name|salary|
+---+--------+------+
| 20|  Manish| 20000|
| 25|  Nikita| 21000|
| 16|  Pritam| 22000|
| 35|Prantosh| 25000|
| 67|  Vikash| 40000|
+---+--------+------+



**NOTE: if we dont set multiline true option will get error**

In [0]:
spark.read.format("json") \
    .option("inferSchema", "true") \
    .option("mode", "permissive") \
    .load("dbfs:/FileStore/shared_uploads/zaderohish5@gmail.com/Multi_line_correct.json").show()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2758707830102862>:1
----> 1 spark.read.format("json") \
      2     .option("inferSchema", "true") \
      3     .option("mode", "permissive") \
      4     .load("dbfs:/FileStore/shared_uploads/zaderohish5@gmail.com/Multi_line_correct.json").show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:920, in DataFrame.show(self, n, truncate, vertical)
    914     raise PySparkTypeError(
    915         error_class="NOT_A_BOOLEAN",
    916         messag

### Which one works faster — multiline or line-delimited?

**Line-Delimited JSON is faster:**
- Spark processes each line as a separate record.
- Optimized for parallel processing and large datasets.

**Multiline JSON is slower:**
- Requires Spark to parse and combine lines into individual records.
- Increases processing overhead.

**When to Use?**
- Use line-delimited JSON for big data processing.
- Use multiline JSON when the input format is nested or human-readable.

### What will happen if I have a corrupted JSON file?
By default, Spark uses PERMISSIVE mode:
- Corrupted records are stored in a column named _corrupt_record.
- The rest of the row is filled with NULL.


In [0]:
spark.read.format("json") \
    .option("inferSchema", "true") \
    .option("mode", "permissive") \
    .load("dbfs:/FileStore/shared_uploads/zaderohish5@gmail.com/corrupted_json.json").show()

+--------------------+----+--------+------+
|     _corrupt_record| age|    name|salary|
+--------------------+----+--------+------+
|                null|  20|  Manish| 20000|
|                null|  25|  Nikita| 21000|
|                null|  16|  Pritam| 22000|
|                null|  35|Prantosh| 25000|
|{"name":"Vikash",...|null|    null|  null|
+--------------------+----+--------+------+



In [0]:
spark.read.format("json") \
    .option("inferSchema", "true") \
    .option("mode", "permissive") \
    .option("multiline", "true") \
    .load("dbfs:/FileStore/shared_uploads/zaderohish5@gmail.com/Multi_line_incorrect.json").show()

# in the json we have multiline josn but its not in list
# we have to list of dictionary(considering one json object as dictionary)

+---+------+------+
|age|  name|salary|
+---+------+------+
| 20|Manish| 20000|
+---+------+------+



### How to convert nested JSON into Spark DataFrame?

In [0]:
df = spark.read.format("json") \
    .option("inferSchema", "true") \
    .option("mode", "permissive") \
    .option("multiline", "true") \
    .load("dbfs:/FileStore/shared_uploads/zaderohish5@gmail.com/nested_json_file.json")

df.show()

+----+-------+--------------------+-------------+-------------+-------------+------+
|code|message|         restaurants|results_found|results_shown|results_start|status|
+----+-------+--------------------+-------------+-------------+-------------+------+
|null|   null|                  []|            0|            0|            1|  null|
|null|   null|[{{{17066603}, b9...|         6835|           20|            1|  null|
|null|   null|                  []|            0|            0|            1|  null|
|null|   null|                  []|            0|            0|            1|  null|
|null|   null|[{{{17093124}, b9...|         8680|           20|            1|  null|
|null|   null|                  []|            0|            0|            1|  null|
|null|   null|                  []|            0|            0|            1|  null|
|null|   null|[{{{17580142}, b9...|          943|           20|            1|  null|
|null|   null|                  []|            0|            0|  

In [0]:
display(df)

code message restaurants results_found results_shown results_start status null null List() 0 0 1 null null null List(List(List(List(17066603), b90e6a8c738410315a20c449fe2eb1b1, 25, Southern, Cajun, Soul Food, $, zomato://restaurant/17066603, List(), https://www.zomato.com/orlando/the-coop-winter-park/events#tabtop?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1, https://b.zmtcdn.com/data/pictures/3/17066603/3002ed08cd3035bae040503e26b19f93_featured_v2.jpg, 0, 0, 17066603, 0, List(610 W Morse Boulevard, Winter Park, FL 32789, Orlando, 601, 216, 28.5973660000, Winter Park, Winter Park, Orlando, -81.3572190000, 32789), https://www.zomato.com/orlando/the-coop-winter-park/menu?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1&openSwipeBox=menu&showMinimal=1#tabtop, The Coop, List(), https://www.zomato.com/orlando/the-coop-winter-park/photos?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1#tabtop, 2, 0, https://b.zmtcdn.com/data/pictures/3/17066603/3002ed08cd3035bae040503e26b19f93_featured_v2.jpg, https://www.zomato.com/orlando/the-coop-winter-park?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1, List(3.6, 9ACD32, Good, 432))), List(List(List(17059541), b90e6a8c738410315a20c449fe2eb1b1, 50, Italian, $, zomato://restaurant/17059541, List(), https://www.zomato.com/orlando/maggianos-little-italy-i-drive-universal/events#tabtop?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1, https://b.zmtcdn.com/data/pictures/1/17059541/50ead37fbc2663d80400c56da8502148_featured_v2.jpg, 0, 0, 17059541, 0, List(9101 International Drive,Orlando, FL 32819, Orlando, 601, 216, 28.4332350000, I-Drive/Universal, I-Drive/Universal, Orlando, -81.4714470000, 32819), https://www.zomato.com/orlando/maggianos-little-italy-i-drive-universal/menu?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1&openSwipeBox=menu&showMinimal=1#tabtop, Maggiano's Little Italy, List(), https://www.zomato.com/orlando/maggianos-little-italy-i-drive-universal/photos?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1#tabtop, 4, 0, https://b.zmtcdn.com/data/pictures/1/17059541/50ead37fbc2663d80400c56da8502148_featured_v2.jpg, https://www.zomato.com/orlando/maggianos-little-italy-i-drive-universal?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1, List(4.4, 5BA829, Very Good, 886))), List(List(List(17064405), b90e6a8c738410315a20c449fe2eb1b1, 10, Asian, Latin American, Vegetarian, $, zomato://restaurant/17064405, List(), https://www.zomato.com/orlando/tako-cheena-by-pom-pom-orlando/events#tabtop?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1, https://b.zmtcdn.com/data/pictures/5/17064405/d9046cbf96ed14c27ccb43d67b2d9e14_featured_v2.jpg, 0, 0, 17064405, 0, List(932 North Mills Avenue, Orlando, FL 32803, Orlando, 601, 216, 28.5578450000, Mills 50, Mills 50, Orlando, -81.3645470000, 32803), https://www.zomato.com/orlando/tako-cheena-by-pom-pom-orlando/menu?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1&openSwipeBox=menu&showMinimal=1#tabtop, Tako Cheena by Pom Pom, List(), https://www.zomato.com/orlando/tako-cheena-by-pom-pom-orlando/photos?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1#tabtop, 1, 0, https://b.zmtcdn.com/data/pictures/5/17064405/d9046cbf96ed14c27ccb43d67b2d9e14_featured_v2.jpg, https://www.zomato.com/orlando/tako-cheena-by-pom-pom-orlando?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1, List(4.4, 5BA829, Very Good, 570))), List(List(List(17057797), b90e6a8c738410315a20c449fe2eb1b1, 40, Mediterranean, Turkish, $, zomato://restaurant/17057797, List(), https://www.zomato.com/orlando/bosphorous-turkish-cuisine-winter-park/events#tabtop?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1, https://b.zmtcdn.com/data/pictures/7/17057797/6504549ba9a80220f1d891b4fc0f0e72_featured_v2.jpg, 0, 0, 17057797, 0, List(108 S Park Ave, Winter Park, FL 32789, Orlando, 601, 216, 28.5976271000, Winter Park, Winter Park, Orlando, -81.3508344000, 32789), https://www.zomato

In [0]:
df.printSchema()

root
 |-- code: long (nullable = true)
 |-- message: string (nullable = true)
 |-- restaurants: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- restaurant: struct (nullable = true)
 |    |    |    |-- R: struct (nullable = true)
 |    |    |    |    |-- res_id: long (nullable = true)
 |    |    |    |-- apikey: string (nullable = true)
 |    |    |    |-- average_cost_for_two: long (nullable = true)
 |    |    |    |-- cuisines: string (nullable = true)
 |    |    |    |-- currency: string (nullable = true)
 |    |    |    |-- deeplink: string (nullable = true)
 |    |    |    |-- establishment_types: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- events_url: string (nullable = true)
 |    |    |    |-- featured_image: string (nullable = true)
 |    |    |    |-- has_online_delivery: long (nullable = true)
 |    |    |    |-- has_table_booking: long (nullable = true)
 |    |    |    |-- i